In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv")
y_train = pd.read_csv("../input/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv")

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
y_train.head()

In [ ]:
y_train.shape

In [ ]:
test = pd.read_csv("../input/Pump_it_Up_Data_Mining_the_Water_Table_-_Test_set_values.csv")

In [ ]:
test.head()

In [ ]:
train = pd.merge(train, y_train, how= 'inner', on='id')

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
import h2o

In [ ]:
h2o.init()

In [ ]:
data_train = h2o.H2OFrame(train)

In [ ]:
data_test = h2o.H2OFrame(test)

In [ ]:
y = 'status_group'
x = data_train.names

In [ ]:
x.remove(y)

In [ ]:
train, test = data_train.split_frame([0.8])

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
mA = H2OAutoML(max_runtime_secs=300)
mA.train(x,"status_group",data_train)

In [ ]:
predict_automl = mA.leader.predict(test)

In [ ]:
predict_automl

In [ ]:
mA.leader.model_performance(test)

In [ ]:
mA.leaderboard

In [ ]:
predict_automl.types

In [ ]:
predict_automl = mA.leader.predict(data_test)

In [ ]:
predict_automl

In [ ]:
autoML_df = predict_automl.as_data_frame(use_pandas=True)

In [ ]:
autoML_df['predict'].value_counts()

In [ ]:
submission = pd.read_csv("../input/Pump_it_Up_Data_Mining_the_Water_Table_-_Submission_format.csv")

In [ ]:
submission.head()

In [ ]:
submission = submission.drop('status_group', axis=1)


In [ ]:
submission.head()

In [ ]:
autoML_df.head()

In [ ]:
submission['status_group'] = autoML_df['predict']

In [ ]:
submission.head()

In [ ]:
submission.to_csv('sub.csv', index = False)

In [ ]:


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "sub.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# create a link to download the dataframe
create_download_link(submission)

# ↓ ↓ ↓  Yay, download link! ↓ ↓ ↓ 

